经过了第四讲的铺垫，在本讲中，我们来正式书写自注意力模块

In [5]:
import torch
import torch.nn as nn  
import torch.nn.functional as F

我们能够看出，目前这种方式只是简单实现过去的编码和当前的编码的十分简单的加权和，也即权重都是通过数量的平均获得，在此之前的每一个字符的编码的权重都是一样的，如果**不同的字符可以根据其自身的情况，根据其自身可以发挥的不同作用**从而生成不同权重的话，，， 没错，这就是自注意力机制做的事情

- 自注意力机制可以被简单地描述为这么一回事
     - 对于每一个被编码的位置，其会有两个向量，一个我们称之为`key`,而另一个我们称之为`query`
     - 我们可以想象，对于每一个`query`，他一直在说**我在寻找什么**
     - 而对于每一个`key`，他一直在说**我是什么，我所包含的是什么**

- 我们在不同序列之间获取联系的方式就是通过`key`和`query`之间的交互，更具体一点来讲，即通过一个点乘，在`key`和`query`之间实现

- 而`key`和`query`之间的点积就变成了权重`weight`

In [ ]:
# version 4: 自注意力的实现
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)


# 一个简单的自注意力头的实现
head_size = 16  # 指定头的大小
key = nn.Linear(C,head_size,bias = False)
query = nn.


trils = torch.tril(torch.ones(T,T))
weight = torch.zeros((T,T))  # 构造一个全为0的向量
weight = weight.masked_fill(trils == 0,float('-inf'))  # 使所有tril为0的位置都变为无穷大
# 然后，我们选择在每行的维度上去使用sotfmax，
weight = F.softmax(weight,dim=-1)

out = weight @ x

out.shape